
# Tutorial: Randomized Search CV: The Efficient Search

Selamat datang di subchapter 3.3! Kita baru saja melihat bagaimana `GridSearchCV` secara teliti mencoba setiap kombinasi hyperparameter. Meskipun sangat kuat, pendekatan ini bisa menjadi sangat lambat jika kita memiliki banyak hyperparameter atau banyak nilai untuk diuji.

Di sinilah **Randomized Search CV** bersinar. Daripada mencoba *semua* kemungkinan, `RandomizedSearchCV` akan mengambil **sampel acak** sejumlah kombinasi tertentu dari ruang pencarian. Pendekatan ini seringkali menemukan solusi yang sangat baik (atau bahkan yang terbaik) dalam waktu yang jauh lebih singkat.

Di notebook ini, kita akan belajar cara mengimplementasikan `RandomizedSearchCV` dan memahami mengapa ini adalah pilihan yang sangat populer di kalangan praktisi *machine learning*.




---
### 1. Tujuan Pembelajaran

Di akhir notebook ini, Anda akan dapat:

* Menjelaskan perbedaan fundamental antara Grid Search dan Randomized Search.
* Mendefinisikan ruang pencarian hyperparameter menggunakan distribusi (bukan hanya daftar).
* Mengimplementasikan `RandomizedSearchCV` untuk menemukan kombinasi hyperparameter yang baik secara efisien.
* Memahami parameter kunci `n_iter` yang mengontrol trade-off antara waktu pencarian dan ketelitian.




---
### 2. Analogi: Koki yang Efisien

Ingat koki perfeksionis kita dari subchapter sebelumnya yang mencoba setiap kombinasi garam dan merica? Sekarang bayangkan seorang **koki yang efisien**.

Koki ini memiliki lebih banyak "tombol" untuk diatur:
1.  **Tingkat Garam:** Antara 1 hingga 5 sendok teh.
2.  **Tingkat Merica:** Antara 0.5 hingga 2 sendok teh.
3.  **Suhu Memasak:** Antara 150°C hingga 220°C.
4.  **Waktu Memasak:** Antara 20 hingga 60 menit.

Mencoba setiap kemungkinan akan memakan waktu berhari-hari! Sebaliknya, koki yang efisien ini memutuskan: "Saya hanya punya waktu untuk membuat **10 versi percobaan**."

Dia kemudian secara **acak** memilih 10 kombinasi dari rentang di atas (misalnya, Garam 2.3 sdt, Merica 1.2 sdt, Suhu 185°C, Waktu 45 menit) dan memilih yang terbaik dari 10 percobaan tersebut.

Inilah ide di balik **Randomized Search**: ia tidak menjelajahi seluruh "grid", melainkan mengambil sampel acak dari dalamnya.




---
### 3. Setup: Data dan Model dengan Banyak Hyperparameter

Untuk melihat kekuatan `RandomizedSearchCV`, kita akan menggunakan model yang lebih kompleks seperti `RandomForestRegressor`, yang memiliki banyak hyperparameter untuk di-*tuning*. Kita masih akan menggunakan dataset California Housing.



In [5]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from scipy.stats import randint

In [2]:
# data
housing = fetch_california_housing()
X,y = housing.data,housing.target
X_train,X_test, y_train, y_test = train_test_split(X,y,random_state=42)

/usr/local/lib/python3.11/site-packages/sklearn/datasets/_base.py:1519: UserWarning: Retry downloading from url: https://ndownloader.figshare.com/files/5976036
  warnings.warn(f"Retry downloading from url: {remote.url}")


HTTPError: HTTP Error 403: Forbidden


---
### 4. Implementasi `RandomizedSearchCV`




#### Langkah 1: Buat Pipeline Model

Langkah ini tetap sama. Kita selalu ingin memasukkan *scaler* ke dalam pipeline.



In [ ]:
pipe = make_pipeline(
	StandardScaler(),
	RandomForestRegressor(random_state=42)
)


#### Langkah 2: Definisikan Distribusi Hyperparameter

Ini adalah perbedaan utama. Daripada memberikan daftar nilai yang tetap, kita bisa memberikan **distribusi** dari mana `RandomizedSearchCV` akan mengambil sampel. Kita akan menggunakan `scipy.stats.randint` untuk mengambil sampel integer secara acak dari rentang tertentu.



In [ ]:
param_dist = {
	'randomforestregressor__n_estimators':randint(50,200), # jumlah pohon
	'randomforestregressor__max_depth': randint(5,30),
	'randomforestregressor__min_samples_leaf': randint(1,10)
}


#### Langkah 3: Inisialisasi `RandomizedSearchCV`

Kita membuat objek `RandomizedSearchCV`. Perhatikan parameter baru `n_iter`.



In [ ]:
random_search = RandomizedSearchCV(
	pipe,
	param_distributions=param_dist,
	n_iter = 20,
	cv=5,
	scoring='r2',
	n_jobs=-1,
	random_state=42
)


#### Langkah 4: Jalankan Pencarian dengan `.fit()`

`RandomizedSearchCV` sekarang akan melakukan 20 percobaan acak.

**Perhitungan:** 20 iterasi × 5 *folds* = **100 kali pelatihan model!**

Ini jauh lebih sedikit daripada jika kita menggunakan `GridSearchCV` pada rentang yang sama, yang bisa mencapai ribuan kali pelatihan.



In [ ]:
random_search.fit(X_train,y_train)

,estimator,Pipeline(step...m_state=42))])
,param_distributions,"{'randomforestregressor__max_depth': <scipy.stats....t 0x134eb5590>, 'randomforestregressor__min_samples_leaf': <scipy.stats....t 0x134eb5810>, 'randomforestregressor__n_estimators': <scipy.stats....t 0x134d0c2f0>}"
,n_iter,20
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan



---
### 5. Menganalisis Hasil

Proses analisisnya sama persis dengan `GridSearchCV`.




#### Mengakses Kombinasi Terbaik



In [ ]:
print(f"Kombinasi hyperparameter terbaik: \n{random_search.best_params_}")
print(f"skor r2 cross-validation terbaik: {random_search.best_score_:.4f}")

Kombinasi hyperparameter terbaik: 
{'randomforestregressor__max_depth': 25, 'randomforestregressor__min_samples_leaf': 2, 'randomforestregressor__n_estimators': 133}
skor r2 cross-validation terbaik: 0.8021



#### Mengakses Model Terbaik



In [ ]:
best_model = random_search.best_estimator_
print("model terbaik yang sudha dilatih:\n",best_model)

model terbaik yang sudha dilatih:
 Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=25, min_samples_leaf=2,
                                       n_estimators=133, random_state=42))])



---
### 6. Evaluasi Akhir pada Test Set



In [ ]:
test_score = best_model.score(X_test,y_test)
print(f"skor r2 pada test set: {test_score:.4f}")

skor r2 pada test set: 0.8083



**Observasi:** Dengan hanya 20 percobaan acak, kita berhasil mendapatkan model Random Forest yang jauh lebih baik (R² ~0.8) daripada model Ridge (R² ~0.6) yang kita *tuning* sebelumnya.




---
### 7. Kesimpulan Tutorial

* **Randomized Search** adalah alternatif yang efisien untuk Grid Search, terutama ketika ruang pencarian hyperparameter sangat besar.
* Parameter `n_iter` mengontrol "anggaran" komputasi Anda—berapa banyak kombinasi yang ingin Anda coba.
* **Kelebihan:** Jauh lebih cepat dan seringkali menemukan solusi yang sangat baik. Memungkinkan Anda untuk menjelajahi rentang hyperparameter yang lebih luas.
* **Kekurangan:** Tidak ada jaminan akan menemukan kombinasi terbaik absolut.

**Aturan Praktis:** Mulailah dengan **Randomized Search** untuk menjelajahi rentang yang luas dan menemukan "wilayah" hyperparameter yang menjanjikan. Kemudian, jika diperlukan, Anda bisa melakukan **Grid Search** di sekitar wilayah yang menjanjikan tersebut untuk penyetelan akhir.
